# Tokenization

In [10]:
from collections import Counter
from typing import List, Tuple, Set
import string
from transformers import AutoTokenizer

## 1. Introduction

Before a model can work with text, we need to turn that text into numbers. Tokenization is the first step in that process — it splits text into smaller pieces (tokens) and assigns each one an ID.

Depending on the tokenizer, a token can be a whole word, a subword, or even a single character. For example, simple tokenizers might just split on spaces, while more advanced ones (like BPE or WordPiece) learn which parts of words are common and reuse them to handle new words.

The goal is to convert text into a form that’s consistent, compact, and meaningful for a model to process. Tokenization might seem simple, but it affects almost everything downstream — vocabulary size, model efficiency, and how well the model handles unseen words.

In general, a **tokenizer** is defined as a tuple $(\Sigma_0, \Sigma, f)$, where
1. $\Sigma_0$ is a (finite) **base vocabulary** describing the minimal units from which strings are built.
2. $\Sigma$ is a (finite) **token vocabulary** that the tokenizer uses to represent outputs.
3. $f: \Sigma_0^* \to \Sigma^*$ is a **tokenization function** that takes strings represented as sequences in the base vocabulary and transforms them into tokens.

Here, we define $\Sigma^*$ as the set of all strings that can be made from a vocabulary $\Sigma$.

## 2. Byte Pair Encoding

Byte Pair Encoding (BPE) is a subword tokenization method that balances between word-level and character-level tokenization. It starts with all characters in the text as individual tokens and then repeatedly merges the most frequent pairs of tokens. Over time, this builds up a vocabulary of common subwords.

The key idea is:
- Frequent character pairs (like t + h → th, then th + e → the) get merged first.
- Rare words can still be represented by smaller subword pieces (like un, ##seen, ##ly).

This helps models handle unknown or rare words better, while keeping the vocabulary size manageable.

In practice, BPE is widely used in modern NLP models (like GPT or RoBERTa). It provides a nice compromise:
- **Efficient:** Fewer tokens than character-level.
- **Flexible:** Can represent unseen words.
- **Consistent:** Keeps common patterns intact.

### Algorithm 1: Dictionary learning for byte pair encoding

**Input:**  
Base vocabulary $\Sigma_0$, dataset $X$, target vocabulary size $ V $

**Output:**  
Learned vocabulary $\Sigma$, merge history $M$


**Initialize**  
$\Sigma \leftarrow \Sigma_0$  
$M \leftarrow []$

**while** $|\Sigma| < V$ **do**  
$\quad (a^*, b^*) = \arg\max_{a,b \in \Sigma}
\sum_{\mathbf{x} \in X} \sum_{i=1}^{|\mathbf{x}|-1}
\mathbf{1}[x_i = a \land x_{i+1} = b]$  
$\quad \text{new\_tok} \leftarrow a^* b^*$  
$\quad \Sigma \leftarrow \Sigma \cup \{\text{new\_tok}\}$  
$\quad M \leftarrow M + [(\text{new\_tok}, a^*, b^*)]$  
$\quad$**for each** $\mathbf{x} \in X$ **do**  
$\qquad \mathbf{x} \leftarrow \text{replace}([a^*, b^*], [\text{new\_tok}], \mathbf{x}) $  
$\quad$**end for**  
**end while**

**return** $ \Sigma, M $

To implement this algorithm, we first write some helper functions

In [2]:
def count_pair_frequencies(X: List[List[str]]) -> Counter:
	"""
	Given a dataset X, returns a counter that count how often each adjacent
	pair of symbols appear in the dataset.
	"""
	pair_freq = Counter()
	for x in X:
		for idx in range(len(x) - 1):
			pair_freq[(x[idx], x[idx+1])] += 1

	return pair_freq

In [3]:
def replace_pair(X: List[List[str]], a: str, b: str, new_tok: str) -> List[List[str]]:
	"""
	Given a dataset X, a pair of token (a, b), and the new token new_tok,
	replace every occurrence of the pair (a, b) with new_tok in the dataset.
	"""
	new_X = []
	for x in X:
		new_seq = []
		idx = 0
		while idx < len(x):
			if idx < len(x) - 1 and x[idx] == a and x[idx+1] == b:
				new_seq.append(new_tok)
				idx += 2
			else:
				new_seq.append(x[idx])
				idx += 1
		new_X.append(new_seq)

	return new_X

Now, we are ready to write the main function for our learning algorithm

In [4]:
def learn_bpe(
	X: List[List[str]],
	target_vocab_size: int,
	base_vocab: Set[str] = None,
	verbose=False
) -> Tuple[Set[str], List[Tuple[str, str, str]]]:
	"""
	Dictionary learning for Byte Pair Encoding (BPE).
	
	Args:
		X: dataset, list of lists of symbols (each x in X is a sequence of chars)
		target_vocab_size: desired vocabulary size
		base_vocab: optional initial vocabulary
		verbose: whether to print the merges

	Returns:
		Sigma: final vocabulary
		M: list of merges (new_token, a, b)
	"""
	# initialize base vocab
	if base_vocab is not None:
		Sigma = base_vocab
	else:
		Sigma = set(char for x in X for char in x)
	
	M = []

	while len(Sigma) < target_vocab_size:
		# get the most common pair
		pair_freq = count_pair_frequencies(X)

		a, b = pair_freq.most_common(1)[0][0]
		# define new token and merge
		new_tok = a + b
		X = replace_pair(X, a, b, new_tok)
		# update
		Sigma.add(new_tok)
		M.append((new_tok, a, b))

		if verbose:
			print(f"Merge {len(M)}: ({a}, {b}) -> {new_tok}")

	return Sigma, M

In [5]:
# simple basic vocab
def generate_basic_vocab(include_special: bool=True) -> Set:
	base_vocab = list(
		string.ascii_letters +
		string.digits +
		string.punctuation +
		' '
	)
	if include_special:
		base_vocab += ['<PAD>', '<UNK>', '<BOS>', '<EOS>']

	return set(base_vocab)

In [6]:
base_vocab = generate_basic_vocab()
print(len(base_vocab))

99


In [7]:
# example
X = [
    list("the cat sat on the mat"),
    list("the dog sat on the log"),
    list("the cat chased the dog"),
    list("the dog chased the cat"),
]

Sigma, M = learn_bpe(X, target_vocab_size=111, base_vocab=base_vocab)

print("\nFinal Vocabulary Size:", len(Sigma))
print("Learned merges:")
for new_tok, a, b in M:
    print(f"{a} + {b} -> {new_tok}")


Final Vocabulary Size: 111
Learned merges:
t + h -> th
th + e -> the
the +   -> the 
a + t -> at
at +   -> at 
  + the  ->  the 
o + g -> og
d + og -> dog
the  + c -> the c
the c + at  -> the cat 
s + at  -> sat 
sat  + o -> sat o


### Algorithm 2: Tokenization for byte pair encoding
**Input:**  
Merge history $M$, string $\mathbf{x}$

**Output**  
Tokenized $\mathbf{x}$

**for each** $\text{tok}, a, b \in \Sigma$ **do**  
$\quad$ $\mathbf{x} \leftarrow \text{replace}([a, b], \text{tok}, \mathbf{x})$  
**end for**

In [8]:
def tokenize(x, merge_history):
	X = [x]
	for tok, a, b in merge_history:
		X = replace_pair(X, a, b, tok)

	return X[0]

In [9]:
# example
example_seq = list("The cat ate the fish")
tokenized_seq = tokenize(example_seq, M)
tokenized_seq

['T', 'h', 'e', ' ', 'c', 'at ', 'at', 'e', ' the ', 'f', 'i', 's', 'h']

## 3. Using Existing Tokenizers

Example usage of a tokenizer from Hugging Face's transformers.AutoTokenizer module

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

print(f"Tokenizer loaded with vocab size: {tokenizer.vocab_size}")
print(f"Special tokens: PAD={tokenizer.pad_token_id}, EOS={tokenizer.eos_token_id}")
print(f"Tokenizer output has keys: {list(tokenizer("").keys())}")

Tokenizer loaded with vocab size: 50257
Special tokens: PAD=50256, EOS=50256
Tokenizer output has keys: ['input_ids', 'attention_mask']


In [42]:
print(f"Input ids: {tokenizer(example_seq)['input_ids']}")
print(f"Attention mask: {tokenizer(example_seq)['attention_mask']}")

Input ids: [[51], [71], [68], [220], [66], [64], [83], [220], [64], [83], [68], [220], [83], [71], [68], [220], [69], [72], [82], [71]]
Attention mask: [[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]
